# All American Transfers

In [1]:
import json
import core_constants as cc
import functions as fx
import pandas as pd
import sqlite3 as sql
import recordlinkage
from recordlinkage.index import Full

## Build out DataFrames

In [2]:
SQL = '''SELECT * from Transfers247_Unranked'''
df_247 = (fx.connDBAndReturnDF(SQL)).set_index('IDYR')
df_247.index.name = '247_IDYR'
df_247

,ID,PlayerName,Year,StandardizedPosition,KeyPositionGroup
247_IDYR,,,,,
thomasbrown_nebraska_2012,thomasbrown_nebraska,thomasbrown,2012,LB,2
zaireanderson_nebraska_2012,zaireanderson_nebraska,zaireanderson,2012,LB,2
lamarrwoodley_michigan_2003,lamarrwoodley_michigan,lamarrwoodley,2003,LB,2
prescottburgess_michigan_2003,prescottburgess_michigan,prescottburgess,2003,DB,2
shawncrable_michigan_2003,shawncrable_michigan,shawncrable,2003,DE,2
...,...,...,...,...,...
brettmedforth_troy_2013,brettmedforth_troy,brettmedforth,2013,OL,1
cardelllue_troy_2013,cardelllue_troy,cardelllue,2013,DB,2
zachmoore_troy_2013,zachmoore_troy,zachmoore,2013,LB,2


In [3]:
SQL = '''SELECT ID, PlayerName, Year from UnlinkedAllAmerican'''
df_aa = (fx.connDBAndReturnDF(SQL)).set_index('ID')
df_aa.index.name = 'AllAmerican_ID'
df_aa

,PlayerName,Year
AllAmerican_ID,,
aaroncurry_wakeforest,aaroncurry,2008
adampacmanjones_westvirginia,adampacmanjones,2004
adamsnyder_oregon,adamsnyder,2004
aidanschneider_oregon,aidanschneider,2015
alanzemaitis_pennstate,alanzemaitis,2005
...,...,...
willsmith_ohiostate,willsmith,2003
willsnyderwine_duke,willsnyderwine,2010
zachmiller_arizonastate,zachmiller,2006


### NOTES

> you don't

In [4]:
indexer = recordlinkage.Index()
indexer.add(Full())

In [5]:
pairs = indexer.index(df_aa, df_247)

In [6]:
from functions import YearOther

sumFields = []
c = recordlinkage.Compare()
c.string('PlayerName', 'PlayerName', method='damerau_levenshtein', label='PlayerName')
sumFields.append('PlayerName')
#c.exact('StandardizedPosition', 'StandardizedPosition', label='StandardizedPosition')
#sumFields.append('StandardizedPosition')
#c.exact('KeyPositionGroup', 'KeyPositionGroup', label='KeyPositionGroup')
#sumFields.append('KeyPositionGroup')
c.add(YearOther('Year', 'Year', label='Year'))
sumFields.append('Year')

In [7]:
features = c.compute(pairs, df_aa, df_247)

In [8]:
sum = 0
for field in sumFields:
    sum = sum + features[field]

features['Sum'] = sum / len(sumFields)

features = features[features['Year'] == 1.0]

features

PlayerName  Year  \
AllAmerican_ID        247_IDYR                                              
aaroncurry_wakeforest lamarrwoodley_michigan_2003          0.307692   1.0   
                      prescottburgess_michigan_2003        0.200000   1.0   
                      shawncrable_michigan_2003            0.181818   1.0   
                      jimpresley_michigan_2003             0.100000   1.0   
                      ryanmundy_michigan_2003              0.300000   1.0   
...                                                             ...   ...   
zachvonrosenberg_lsu  rogiertenlohuis_georgiastate_2016    0.062500   1.0   
                      tjarnold_georgiastate_2016           0.125000   1.0   
                      jaquezparks_georgiastate_2016        0.125000   1.0   
                      terrancedixon_georgiastate_2016      0.125000   1.0   
                      malikbesseck_georgiastate_2016       0.187500   1.0   

                                                              Sum  
AllAmerican_ID        247_IDYR                                     
aaroncurry_wakeforest lamarrwoodley_michigan_2003        0.653846  
                      prescottburgess_michigan_2003      0.600000  
                      shawncrable_michigan_2003          0.590909  
                      jimpresley_michigan_2003           0.550000  
                      ryanmundy_michigan_2003            0.650000  
...                                                           ...  
zachvonrosenberg_lsu  rogiertenlohuis_georgiastate_2016  0.531250  
                      tjarnold_georgiastate_2016         0.562500  
                      jaquezparks_georgiastate_2016      0.562500  
                      terrancedixon_georgiastate_2016    0.562500  
                      malikbesseck_georgiastate_2016     0.593750  

[946566 rows x 3 columns]

In [9]:
features.insert(0, 'sourceID', features.index.get_level_values(0))
features.insert(1, 'targetID', features.index.get_level_values(1))

filteredList = []
for idx, data in features.groupby(level=0):
    data = data.loc[data['Sum'].idxmax()]
    #.94
    if (data['Sum'] >.95):
        filteredList.append(data)
dfFinal = pd.DataFrame()
dfFinal = dfFinal.append(filteredList)
dfFinal.to_csv("resultsAllAmericanTransfer.csv")

In [10]:
dfFinal

,sourceID,targetID,PlayerName,Year,Sum
"(alihighsmith_lsu, alihighsmith_miami_2003)",alihighsmith_lsu,alihighsmith_miami_2003,1.0,1.0,1.0
"(anthonyfera_texas, anthonyfera_pennstate_2009)",anthonyfera_texas,anthonyfera_pennstate_2009,1.0,1.0,1.0
"(arthurbrown_kansasstate, arthurbrown_miami_2008)",arthurbrown_kansasstate,arthurbrown_miami_2008,1.0,1.0,1.0
"(chasebeeler_stanford, chasebeeler_iowastate_2006)",chasebeeler_stanford,chasebeeler_iowastate_2006,1.0,1.0,1.0
"(chrismiller_ballstate, chrismiller_oregonstate_2004)",chrismiller_ballstate,chrismiller_oregonstate_2004,1.0,1.0,1.0
"(jarvisjones_georgia, jarvisjones_lsu_2007)",jarvisjones_georgia,jarvisjones_lsu_2007,1.0,1.0,1.0
"(jjwatt_wisconsin, jjwatt_centralmichigan_2007)",jjwatt_wisconsin,jjwatt_centralmichigan_2007,1.0,1.0,1.0
"(johnsullivan_newmexico, johnsullivan_notredame_2003)",johnsullivan_newmexico,johnsullivan_notredame_2003,1.0,1.0,1.0
"(jordanwhite_westernmichigan, jordanwhite_floridainternational_2010)",jordanwhite_westernmichigan,jordanwhite_floridainternational_2010,1.0,1.0,1.0
"(joshdoctson_tcu, joshdoctson_wyoming_2011)",joshdoctson_tcu,joshdoctson_wyoming_2011,1.0,1.0,1.0


In [11]:
fuzzyMI = pd.MultiIndex.from_frame(dfFinal)
recordlinkage.write_annotation_file(
    "../Annotations/Annotations/annotation_AATransfers.json",
    fuzzyMI[0:200],
    df_aa,
    df_247,
    dataset_a_name="All American",
    dataset_b_name="Master"
)

In [12]:
annotation = recordlinkage.read_annotation_file("..//Annotations//Results//result-AAUnranked.json")
try:
    annotation_dict = (annotation.links).to_flat_index()
except Exception as e:
    print(e)

In [13]:
for record in annotation_dict:
    #MAKE SURE YOU UPDATE THE THIRD VALUE TO THE CORRECT KEYDATASET!!
    Values = [record[0], record[1], 6, 1, 1, 1]
    query = '''INSERT INTO RecordLinks(MasterID, TargetID, KeyDataSet, KeyLinkType, LinkConfidence, Transfer)
        VALUES (?,?,?,?,?,?)'''
    
    conn = sql.connect(cc.databaseName)
    c = conn.cursor()
    
    c.execute(query, Values)
    conn.commit()
    
conn.close()